In [44]:
#Celda para librerías
import sklearn as sk

import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
# Modelos base de Machine Learning que vamos a utilizar para hacer pruebas iniciales
#La métrica usada para medir los modelos será el AUC-ROC
# LightGBM
from lightgbm import LGBMClassifier
#KGBoost
from xgboost import XGBClassifier
#KNN
from sklearn.neighbors import KNeighborsClassifier
#Neural Network
from sklearn.neural_network import MLPClassifier
#Random Forest
from sklearn.ensemble import RandomForestClassifier


import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())



# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)


#Importando los datos

app_train = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/application_test.csv')
"""
bureau = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/bureau_balance.csv')
credit_card_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/credit_card_balance.csv')
installments_payments = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/installments_payments.csv')
pos_cash_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/POS_CASH_balance.csv')
previous_application = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/previous_application.csv')
"""
#info = app_train.info()
#print(info)
#describe = app_train.describe()

1.23.5
Directorio actual: c:\Users\Yeray\Desktop\DATA_SCIENCE_ML\Home-Credit-TFG\JUPYTER_NOTEBOOKS


"\nbureau = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/bureau.csv')\nbureau_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/bureau_balance.csv')\ncredit_card_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/credit_card_balance.csv')\ninstallments_payments = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/installments_payments.csv')\npos_cash_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/POS_CASH_balance.csv')\nprevious_application = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/previous_application.csv')\n"

In [45]:
#Feature Engineering


app_train.drop(columns=['SK_ID_CURR'], inplace=True)
#Debido a un fallo que detecta que todas los valores de la columna DAYS_BIRTH 
# son nan cuando no es así, vamos a sustituir la columna por una nueva llamada AGE_INT
app_train['AGE_INT'] = app_train['DAYS_BIRTH']/-365
app_train.drop(columns=['DAYS_BIRTH'], inplace=True)


#Vamos a convertir la variable CNT_CHILDREN en una variable categórica que indique 1 si tiene hijos y 0 si no.
app_train['HAS_CHILDREN'] = np.where(app_train['CNT_CHILDREN'] > 0, 1, 0)
app_train.drop(columns=['CNT_CHILDREN'], inplace=True)
#Vamos a crear la variable DAYS_EMPLOYED en una variable categórica que indique 1 si tiene empleo y 0 si no.
app_train['HAS_EMPLOYMENT'] = app_train['DAYS_EMPLOYED'].map(lambda x: 1 if x < 0 else 0)

#Voy a crear RATIOS para algunas de las variables más importantes (en mi opinión) y que además tienen valores atípicos.
app_train['CREDIT_AMT_INCOME_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_INCOME_TOTAL']
app_train['ANNUITY_AMT_INCOME_RATIO'] = app_train['AMT_ANNUITY']/app_train['AMT_INCOME_TOTAL']
app_train['CREDIT_ANNUITY_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_ANNUITY']
app_train['CREDIT_GOODS_PRICE_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_GOODS_PRICE']
app_train['GOODS_INCOME_RATIO'] = app_train['AMT_GOODS_PRICE']/app_train['AMT_INCOME_TOTAL']


#EXT_SOURCE son variables representan fuentes externas de información sobre el cliente, y suelen estar relacionadas con puntuaciones de riesgo crediticio generadas por 
# instituciones externas al prestamista, como burós de crédito u otras entidades que evalúan el perfil financiero de los clientes.
#Vamos a crear variables relacionadas con EXT_SOURCE.

app_train['EXT_SOURCE_SUM'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
app_train['EXT_SOURCE_MEAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
app_train['EXT_SOURCE_MEDIAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1)
app_train['EXT_SOURCE_MAX'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)
app_train['EXT_SOURCE_MIN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_train['EXT_SOURCE_WEIGHTED_SUM'] = app_train['EXT_SOURCE_1']*2 + app_train['EXT_SOURCE_2']*3 + app_train['EXT_SOURCE_3']*4
app_train['EXT_SOURCE_WEIGHTED_MEAN'] = (app_train['EXT_SOURCE_1']*2 + app_train['EXT_SOURCE_2']*3 + app_train['EXT_SOURCE_3']*4)/3



In [46]:
# Preprocesamiento previo a la creación de los modelos
# Imputación de valores nulos. Vamos a probar de momento a completar variables numéricos con la media
# y las categóricas con la moda.
# Como sabemos que DAYS_EMPLOYEED tiene un valor erroneo, vamos a reemplazarlo por NaN
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# Algunas variables categóricas tienen valores como XNA, vamos a reemplazarlos por NaN
def reemplazar_xna_por_nan(app_train):
    variables_categoricas = app_train.select_dtypes(include=['object']).columns
    for col in variables_categoricas:
        app_train[col] = app_train[col].replace('XNA', np.nan)
    return app_train
app_train_noxna = reemplazar_xna_por_nan(app_train)

app_train = reemplazar_xna_por_nan(app_train)
def imputar_valores_nulos(app_train):
    variables_continuas = app_train.select_dtypes(include=['int64', 'float64']).columns 
    for col in variables_continuas:
        app_train[col] = app_train[col].fillna(app_train[col].mean())
    variables_categoricas = app_train.select_dtypes(include=['object']).columns
    for col in variables_categoricas:
        app_train[col] = app_train[col].fillna(app_train[col].mode()[0])

    return app_train

app_train_nonan = imputar_valores_nulos(app_train_noxna)
#Comprobamos que no haya valores nulos
#print(app_train['DAYS_BIRTH'].unique())
#print(app_train['DAYS_BIRTH'].isnull().sum())
#print(app_train['DAYS_BIRTH'].dtype)

# Verificar si hay valores nulos en app_train después de la imputación
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan.isnull().sum().sum())

Valores nulos en app_train después de la imputación:
0


In [47]:
# Tratamiento de variables categóricas
# Vamos a probar con Label Encoding para las variables categoricas binarias y con 
# One Hot Encoding para el resto.

variables_categoricas = app_train.select_dtypes(include=['object']).columns
variables_categoricas_binarias = [col for col in variables_categoricas if app_train[col].nunique() == 2]
variables_categoricas_no_binarias = [col for col in variables_categoricas if app_train[col].nunique() > 2]
def label_encoding_binarias(app_train):
    Encoder = LabelEncoder()
    for col in variables_categoricas_binarias:
        app_train[col] = Encoder.fit_transform(app_train[col])
    return app_train

app_train_nonan_le = label_encoding_binarias(app_train_nonan)


def one_hot_encoding_no_binarrias(app_train):
    for col in variables_categoricas_no_binarias:
        app_train = pd.get_dummies(app_train, columns=[col])
    return app_train

app_train_nonan_le_oh = one_hot_encoding_no_binarrias(app_train_nonan_le)
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan_le_oh.isnull().sum().sum())



Valores nulos en app_train después de la imputación:
0


In [48]:
# Normalización de las variables
# Como gracias a la exploración de datos hemos visto que hay variables que ya han sido normalizadas 
# previamente, nos vamos a centrar en aquellas que necesitan ser normalizadas.
variables_continuas = app_train_nonan_le_oh.select_dtypes(include=['int64', 'float64']).columns
variables_normalizadas = [col for col in variables_continuas if
                          app_train[col].min() >= 0 and app_train[col].max() <= 1]
variables_NO_normalizadas = [col for col in variables_continuas if col not in variables_normalizadas]

# Vamos a probar con Log Transformation por el momento debido a que tienen bastantes valores atípicos
# log1p transforma los valores negativos en nan por lo que vamos a usar RobustScaler para evitarlo
scaler = RobustScaler()
#Voy a probar también con MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
def normalización(app_train):
    for col in variables_NO_normalizadas:
        app_train[col] = scaler2.fit_transform(app_train[[col]])
    return app_train

app_train_nonan_le_oh_norm = normalización(app_train_nonan_le_oh)
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan_le_oh_norm.isnull().sum().sum())

Valores nulos en app_train después de la imputación:
0


In [49]:
#Tras este preprocesamiento inicial vamos a probar diferentes algoritmos. La métrica usada será AUC_ROC.
#Vamos a probar con los siguientes algoritmos:LightGBM, XGBoost, KNN, Neural Network y Random Forest
x = app_train_nonan_le_oh_norm.drop(columns=['TARGET'])
y = app_train_nonan_le_oh_norm['TARGET']

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Resultados de los modelos conforme he ido haceindo cambios
#LightGBM
-LightGBM AUC: 0.7536 (Log Transformation)
-LightGBM AUC: 0.7576 (Robust Scaler)
-LightGBM AUC: 0.7577 (MinMax)
-LightGBM AUC: 0.7577 (Se me había pasado reemplazar algunos valores XNA en variables categóricas)
-LightGBM AUC: 0.7578 (Variable HAS_CHILDREN y HAS_EMPLOYMENT añadida. Variable CNT_CHILDREN eliminada)
-LightGBM AUC: 0.7664 (He creado 5 nuevas, RATIOS de algunas variables más "importantes")
-LightGBM AUC: 0.7654 (Tras añadir variables relacionadas con EXT_SOURCE)

#XGBoost
-XGBoost AUC: 0.7491
-XGBoost AUC: 0.7481
-XGBoost AUC: 0.7481
-XGBoost AUC: 0.7488
-XGBoost AUC: 0.7491
-XGBoost AUC: 0.7599
-XGBoost AUC: 0.7588

#Random Forest
-Random Forest AUC: 0.7068
-Random Forest AUC: 0.7136
-Random Forest AUC: 0.7138
-Random Forest AUC: 0.7154
-Random Forest AUC: 0.7097
-Random Forest AUC: 0.7152
-Random Forest AUC: 0.7236

#Extra Trees
-Extra Trees AUC: 0.6981
-Extra Trees AUC: 0.7052
-Extra Trees AUC: 0.7048
-Extra Trees AUC: 0.7038
-Extra Trees AUC: 0.7053
-Extra Trees AUC: 0.7077
-Extra Trees AUC: 0.7262

#Logistic Regresion
-Logistic Regression AUC: 0.7417
-Logistic Regression AUC: 0.7478
-Logistic Regression AUC: 0.7457
-Logistic Regression AUC: 0.7452
-Logistic Regression AUC: 0.7459
-Logistic Regression AUC: 0.7480
-Logistic Regression AUC: 0.7497

#Neural Network
-Neural Network AUC: 0.6861
-Neural Network AUC: 0.7437 (Como la primera vez tardó demasiado lo descarté, pero tras el último cambio, Variable HAS_CHILDREN y HAS_EMPLOYMENT añadida, he probado a ejecutarla e interrumpirla pronto y me ha dado un buen resultado)

#Modelos descartados

#KNN (He decidido no seguir haciendo pruebas con KNN. En comparación con los otros modelos los resultados son más bajos y cada vez que refino el proceso empeora)
-KNN AUC: 0.5482
-KNN AUC: 0.5575
-KNN AUC: 0.5497
-KNN AUC: 0.5471





In [50]:
#LightGBM


# Remove special characters from column names
X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

lgb_model = LGBMClassifier(random_state=42)
lgb_model.fit(X_train, Y_train)
lgb_predictions = lgb_model.predict_proba(X_test)[:, 1]

#Evaluación
lgb_auc = roc_auc_score(Y_test, lgb_predictions)
print(f"LightGBM AUC: {lgb_auc:.4f}")

[LightGBM] [Info] Number of positive: 19876, number of negative: 226132
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.130473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14516
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 240
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080794 -> initscore=-2.431606
[LightGBM] [Info] Start training from score -2.431606
LightGBM AUC: 0.7654


In [51]:
#XGBoost
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, Y_train)
xgb_predictions = xgb_model.predict_proba(X_test)[:, 1]

#Evaluación
xgb_auc = roc_auc_score(Y_test, xgb_predictions)
print(f"XGBoost AUC: {xgb_auc:.4f}")

XGBoost AUC: 0.7588


In [25]:
#Neural Network
nn_model = MLPClassifier(random_state=42)
nn_model.fit(X_train, Y_train)
nn_predictions = nn_model.predict_proba(X_test)[:, 1]

#Evaluación
nn_auc = roc_auc_score(Y_test, nn_predictions)
print(f"Neural Network AUC: {nn_auc:.4f}")

Neural Network AUC: 0.7437


In [52]:
#Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, Y_train)
rf_predictions = rf_model.predict_proba(X_test)[:, 1]

#Evaluación
rf_auc = roc_auc_score(Y_test, rf_predictions)
print(f"Random Forest AUC: {rf_auc:.4f}")

Random Forest AUC: 0.7236


In [53]:
#Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
et_model = ExtraTreesClassifier(random_state=42)
et_model.fit(X_train, Y_train)
et_predictions = et_model.predict_proba(X_test)[:, 1]

#Evaluación
et_auc = roc_auc_score(Y_test, et_predictions) 
print(f"Extra Trees AUC: {et_auc:.4f}")


Extra Trees AUC: 0.7262


In [54]:
#Regresión Logística
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, Y_train)
lr_predictions = lr_model.predict_proba(X_test)[:, 1]

#Evaluación
lr_auc = roc_auc_score(Y_test, lr_predictions)
print(f"Logistic Regression AUC: {lr_auc:.4f}")

Logistic Regression AUC: 0.7497
